# Expressive power testing experiment

In this experiment we will examine the expressive power of non permutation equivariant pre-colorings, specifically random features generation.
We will train GIN model on a relatively small pair of 1-WL indistinguishable graphs using random pre-coloring and the spectral pre-coloring.

The random features make it possible for the GNN to distinguish between the graphs at the cost of distinguishing between two isomorphic graphs. 
This situation will force the GNN to think that it gets a new graph each time and prevent it from learning the diffrences between the two graphs.
The spectral features that are permutation equivariant and strictly more expressive than 1-WL will differentiate between the graphs while generating a constant representatation for each set of isomorphic graphs. 

In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.0+cu116.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.12.0+cu116.html
!pip install torch-geometric
!pip install class-resolver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu116.html
     |████████████████████████████████| 8.0 MB 2.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu116.html
     |████████████████████████████████| 3.5 MB 2.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 407 kB 5.2 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=b40a23ef4a37ed720eab2e553d0e072a978ec92b5566e38a562b23e4a39480f8
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/pub

In [ ]:
import numpy as np
import networkx as nx
from networkx import laplacian_matrix
import torch
from torch_geometric.utils import to_undirected
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GIN,BatchNorm
from torch_scatter import scatter_add
from sklearn.model_selection import StratifiedKFold
from torch import optim
from tqdm import tqdm

In [ ]:
non_isomorphic_graphs = (
    np.array([
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 1], 
        [1, 0, 1, 0, 0, 0, 0, 0, 0, 0], 
        [0, 1, 0, 1, 0, 0, 0, 1, 0, 0], 
        [0, 0, 1, 0, 1, 0, 0, 0, 0, 0], 
        [0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 1, 0, 0, 0], 
        [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
        [0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    ]),
    np.array([
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 1, 0, 0, 0, 0, 1, 0],
        [0, 0, 1, 0, 1, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    ]))

def array2data(array: np.array, x: torch.Tensor, y):
    g = nx.from_numpy_array(array)
    edge_index = torch.tensor(list(g.edges())).T
    if edge_index.size()[0] != 0:
        edge_index = to_undirected(edge_index)
    else:
        edge_index = torch.Tensor([[], []]).long()
    data = Data(x, edge_index, y=y)
    return data
  
def random_permutation(n):
  p = np.random.permutation(np.eye(n))
  return p

def separate_data(dataset, seed, fold_idx):
    assert 0 <= fold_idx < 10, "fold_idx must be from 0 to 9."
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

    idx_list = []
    for idx in skf.split(np.zeros(len(dataset)), np.zeros(len(dataset))):
        idx_list.append(idx)
    train_idx, test_idx = idx_list[fold_idx]

    return list(dataset[i] for i in list(train_idx)), list(dataset[i] for i in list(test_idx))


def add_random_features(dataset) :
       for i in range(len(dataset)):                                                          
        dataset[i].x = torch.rand((dataset[i].x.size(0),10)).to(dataset[i].x.device)


def heat_kernel_features(edge_index: torch.Tensor, num_nodes, t_range=torch.logspace(-2, 2, steps=10)):
      g = nx.Graph()
      g.add_nodes_from(list(range(num_nodes)))
      g.add_edges_from((to_undirected(edge_index).T.tolist()))
      adj_mat = torch.tensor(nx.to_numpy_array(g))
      n = len(adj_mat)
      T = len(t_range)
      t_range = t_range.double() 
      laplacian = torch.Tensor(laplacian_matrix(nx.from_numpy_array(adj_mat.numpy())).toarray())
      eig_val, eig_vec = torch.linalg.eigh(laplacian.double())
      eigen_vec_3d = eig_vec.unsqueeze(0).repeat(n, 1, 1) * (eig_vec.unsqueeze(1).repeat(1, n, 1))
      H = torch.exp(-eig_val.unsqueeze(1) @ t_range.unsqueeze(0)).T @ eigen_vec_3d.transpose(1, 2)
      H = H.transpose(1, 2).transpose(0, 1)
      features = H[range(len(H)), range(len(H))].clone()
      return features.float()

def add_spectral_features(dataset):
    for i in range(len(dataset)):
        dataset[i].x = heat_kernel_features(dataset[i].edge_index, dataset[i].num_nodes)                                                                  

def generate_dataset(graphs, num=500):
    a1, a2 = graphs
    nodes = a1.shape[0]
    permutad_graphs = []
    for _ in range(num):
      p = random_permutation(nodes)
      permutad_graphs += [array2data(p @ a1 @ p.T, x=torch.ones((nodes, 1)), y=0)]
      p = random_permutation(nodes)
      permutad_graphs += [array2data(p @ a2 @ p.T, x=torch.ones((nodes, 1)), y=1)]

    return permutad_graphs

###pass data to model with minibatch during testing to avoid memory overflow (does not perform backpropagation)
def pass_data_iteratively(model, graphs, batch_size=256):
    model.eval()
    output = []
    labels = []
    loader = DataLoader(graphs, batch_size=batch_size, shuffle=True)
    for data in loader:
        output.append(model(data).detach())
        labels.append(data.y)
    return torch.cat(output, 0), torch.cat(labels, 0)

def train(batch_size, model, train_graphs, optimizer):
    model.train()
    criterion = torch.nn.CrossEntropyLoss()

    train_graphs_loader = DataLoader(train_graphs, batch_size=batch_size, shuffle=True)
    for batch_graphs in train_graphs_loader:
        output = model(batch_graphs)
        labels = batch_graphs.y.to(batch_graphs.x.device)
        # compute loss
        loss = criterion(output, labels)

        # backprop
        if optimizer is not None:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

def test(model, test_graphs):
    model.eval()

    output, labels = pass_data_iteratively(model, test_graphs)
    pred = output.max(1, keepdim=True)[1]
    correct = pred.eq(labels.view_as(pred).to(output.device)).sum().item()
    acc_test = correct / float(len(test_graphs))

    return acc_test

In [ ]:
class MyGNN(torch.nn.Module):
    def __init__(self, in_channels=10, hidden_channels=64, num_layers=2, num_classes=2):
        super().__init__()
        self.inner_model = GIN(in_channels, hidden_channels, num_layers, out_channels=None,
                                     norm=BatchNorm(hidden_channels))
        self.linear = torch.nn.Linear(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch.to(data.x.device)
        node_desciptors = self.inner_model(x, edge_index)
        graph_descriptors = scatter_add(node_desciptors, batch, dim=0)
        return self.linear(graph_descriptors)

In [ ]:
repeats = 10
accs_spectral = np.zeros(repeats)
accs_random = np.zeros(repeats)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
for r in range(repeats):
      dataset = generate_dataset(non_isomorphic_graphs)
      for i in range(len(dataset)):
          dataset[i] = dataset[i].to(device)
      train_graphs, test_graphs = separate_data(dataset, seed=0, fold_idx=0)

      add_random_features(train_graphs)
      add_random_features(test_graphs)
      model = MyGNN().to(device)
      optimizer = optim.Adam(model.parameters(), lr=0.01)

      for epoch in tqdm(range(0, 50)):
          train(256, model, train_graphs, optimizer)

      accs_random[r] = test(model, test_graphs)


      add_spectral_features(train_graphs)
      add_spectral_features(test_graphs)
      for i in range(len(dataset)):
          dataset[i] = dataset[i].to(device)
      model = MyGNN().to(device)
      optimizer = optim.Adam(model.parameters(), lr=0.01)
     

      for epoch in tqdm(range(0, 50)):
          train(256, model, train_graphs, optimizer)

      accs_spectral[r] = test(model, test_graphs)

print()
print( f"Average accuracy for random features:{accs_random[0].mean()}±{accs_random[0].std()}")
print( f"Average accuracy for spectral features: {accs_spectral[0].mean()}±{accs_spectral[0].std()}")

100%|██████████| 50/50 [00:03<00:00, 13.56it/s]


Average accuracy for random features:0.5±0.0
Average accuracy for spectral features: 1.0±0.0
